In [11]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [12]:
# Login
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('brandonwright@utexas.edu', 'wLXEQ8ffrB')

if not status:
    print(msg)
else:
    print('You are logged in!')

You are logged in!


In [124]:
def get_formatted_dataframe(xls_file, year):
    df = pd.read_excel(xls_file)
    df = df[['DZRATING', 'DZCAMPUS','DPETALLC','DA0AT' + str(year) + 'R', 'DA0912DR' + str(year) + 'R','DAGC4X' + str(year) + 'R','DAGC5X' + str(year - 1) + 'R','DA0GR' + str(year) + 'N','DA0CT' + str(year) + 'R','DA0CC' + str(year) + 'R','DA0CSA' + str(year) + 'R','DA0CAA' + str(year) + 'R','DPSTTOSA','DPSTEXPA','DPFRAALLT','DPFRAALLK','DPFRASTAP','DZRVLOCP','DPFRAFEDP','DPFEAINST','DPFEAINSK','DISTSIZE','COMMTYPE', 'PROPWLTH', 'TAXRATE']]
    df = df.rename(index=str, columns = {"DZRATING":"rating", "DZCAMPUS":"num_schools", "DPETALLC":"num_students", "DA0AT" + str(year) + "R":"attendance_rate", "DA0912DR" + str(year) + "R":"dropout_rate", "DAGC4X" + str(year) + "R":"grad_rate_4_year", "DAGC5X" + str(year - 1) + "R":"grad_rate_5_year", "DA0GR" + str(year) + "N":"annual_grad_count", "DA0CT" + str(year) + "R":"college_admissions_per_tested", "DA0CC" + str(year) + "R":"college_admissions_at_crit", "DA0CSA" + str(year) + "R":"average_sat", "DA0CAA" + str(year) + "R":"average_act", "DPSTTOSA":"average_teacher_salary", "DPSTEXPA":"average_teacher_exp", "DPFRAALLT":"total_revenue", "DPFRAALLK":"total_revenue_per_pupil", "DPFRASTAP":"percent_revenue_from_state", "DZRVLOCP":"percent_revenue_from_local", "DPFRAFEDP":"percent_revenue_from_federal", "DPFEAINST":"instr_expenditures", "DPFEAINSK":"instr_expenditures_per_pupil", "DISTSIZE":"district_size", "COMMTYPE":"community_type", "PROPWLTH":"property_wealth", "TAXRATE":"tax_rate"})
    return df

In [178]:
test_df = get_formatted_dataframe('2016-2017.xls', 16)
train_df = get_formatted_dataframe('2015-2016.xls',15)
train_df = train_df.append(get_formatted_dataframe('2014-2015.xls',14))
train_df = train_df.append(get_formatted_dataframe('2013-2014.xls',13))
print('Test dataframe shape:', test_df.shape)
print('Train dataframe shape:', train_df.shape)

Test dataframe shape: (1203, 25)
Train dataframe shape: (3653, 25)


In [179]:
def clean_dataframe(df):
    df = df.loc[(df['rating'] == "Met Standard") | (df['rating'] == "Met Alternative Standard") |(df['rating'] == "Improvement Required")]
    features = list(df)[3:21]
    for feature in features:
        df = df.loc[(df[feature] != '.')]
    df = df.dropna()
    return df

In [180]:
# Cleaning the data
train_df = clean_dataframe(train_df)
test_df = clean_dataframe(test_df)
print('After cleaning:')
print('Test dataframe shape:', test_df.shape)
print('Train dataframe shape:', train_df.shape)

After cleaning:
Test dataframe shape: (908, 25)
Train dataframe shape: (2749, 25)


In [182]:
# Dealing with class imbalance

print("Before oversampling:")
print(train_df['rating'].value_counts())
print()

from imblearn.over_sampling import SMOTENC
ad = SMOTENC(categorical_features=[20, 21, 22, 23], random_state=None)
train_df_y = train_df['rating']
train_df_x = train_df.iloc[:,1:]
x_res, y_res = ad.fit_sample(train_df_x, train_df_y)

train_df = pd.DataFrame(data=x_res)
train_df['rating'] = y_res

print("After oversampling:")
print(train_df['rating'].value_counts())

Before oversampling:
Met Standard                2589
Improvement Required         107
Met Alternative Standard      53
Name: rating, dtype: int64

After oversampling:
Met Alternative Standard    2589
Met Standard                2589
Improvement Required        2589
Name: rating, dtype: int64


In [16]:
test_df.to_csv('test_data.csv')
train_df.to_csv('train_data.csv')

In [17]:
TRAIN_DATASET='train_data.csv'
TEST_DATASET = 'test_data.csv'
train_df = pd.read_csv('train_data.csv')
test_df = pd.read_csv('test_data.csv')
status, message = ds.delete_dataset(TRAIN_DATASET)
status, dataset = ds.upload_dataset( TRAIN_DATASET)
if not status:
    print(dataset)

In [18]:
#clean train_Set
target = "rating"
status, job_id = ds.clean_data(TRAIN_DATASET, target = target)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-22T00:14:53.625938', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['train_data.csv'], 'artifact_names': ['2e6729ef06894636a6ff4c0150752bd5'], 'model_name': None, 'job_error': None}
{'status': 'Complete', 'starttime': '2019-04-22T00:14:53.625938', 'endtime': '2019-04-22T00:14:57.026071', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['train_data.csv'], 'artifact_names': ['2e6729ef06894636a6ff4c0150752bd5'], 'model_name': None, 'job_error': ''}


In [ ]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])


In [183]:
feature_importance[:10]

NameError: name 'feature_importance' is not defined

In [ ]:
status, artifact = ds.run_model(UPSAMPLE_TRAIN_DATASET, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

In [ ]:
unq = prediction[target].unique()[::-1]
p = np.zeros((len(prediction),))
a = np.zeros((len(prediction),))
for i,q in enumerate(unq):
    p += i*(prediction[target] == q).values
    a += i*(upsample_train_df[target] == q).values
#Plot predictions vs actual
plt.plot(a)
plt.plot(p)
plt.legend(['Actual','Predicted'])
plt.yticks([i for i in range(len(unq))],[q for q in unq]);
print(classification_report(upsample_train_df[target], prediction[target]))

In [ ]:
#TEST DATASET
status, message = ds.delete_dataset(TEST_DATASET)
status, dataset = ds.upload_dataset( TEST_DATASET)
if not status:
    print(dataset)

In [ ]:
status, job_id = ds.clean_data(TEST_DATASET, target = target)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
dataset = ds.download_dataset(job_id['artifact_name'], artifact_path='/Users/ctjoe/Documents/CS363D/darwin/darwin-sdk/sc_proj')
print(dataset)

In [ ]:
clean_test_df = pd.read_csv(dataset[1]['filename'])
print(clean_test_df.head())
clean_test_df['rating'] = 'N/A'
clean_test_df.loc[clean_test_df['rating$$Met Standard'] == 1.0, 'rating'] = 1.0
clean_test_df.loc[clean_test_df['rating$$Met Alternative Standard'] == 1.0, 'rating'] = 2.0
clean_test_df.loc[clean_test_df['rating$$Improvement Required'] == 1.0, 'rating'] = 0
labels = list(clean_test_df)
labels.remove('rating$$Improvement Required')
labels.remove('rating$$Met Alternative Standard')
labels.remove('rating$$Met Standard')
clean_test_df = clean_test_df[labels]
labels.remove('rating')
clean_test_df.loc[clean_test_df['rating'] == 1.0, 'rating'] = 'Met Standard'
clean_test_df.loc[clean_test_df['rating'] == 2.0, 'rating'] = 'Met Alternative Standard'
clean_test_df.loc[clean_test_df['rating'] == 0.0, 'rating'] = 'Improvement Required'


In [ ]:
clean_test_df.to_csv('modified_test_data.csv')
MODIFIED_TEST_DATASET = 'modified_test_data.csv'
modified_test_df = pd.read_csv('modified_test_data.csv')
status, message = ds.delete_dataset(MODIFIED_TEST_DATASET)
status, dataset = ds.upload_dataset(MODIFIED_TEST_DATASET)
if not status:
    print(dataset)

In [ ]:

status, job_id = ds.clean_data(MODIFIED_TEST_DATASET, target = target, model_name = model)
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

In [ ]:
status, artifact = ds.run_model(MODIFIED_TEST_DATASET, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

In [ ]:
# Create plots comparing predictions with actual target
status, prediction = ds.download_artifact(artifact['artifact_name'])
df = pd.read_csv(MODIFIED_TEST_DATASET)
unq = prediction[target].unique()[::-1]
p = np.zeros((len(prediction),))
a = np.zeros((len(prediction),))
for i,q in enumerate(unq):
    p += i*(prediction[target] == q).values
    a += i*(df[target] == q).values
#Plot predictions vs actual
plt.plot(a)
plt.plot(p)
plt.legend(['Actual','Predicted'])
plt.yticks([i for i in range(len(unq))],[q for q in unq]);
print(classification_report(df[target], prediction[target]))